## テンプレート

In [ ]:
import cv2
#画像を読み込む。第二引数は 0:グレースケール、1:カラー画像 -1:カラー＋アルファ
img = cv2.imread('Images/arfukuoka.png',1)
#画像をWindowで表示
cv2.imshow('image',img)
#キー入力を待つ
cv2.waitKey(0)
#何かしらのキーを打つとウィンドウを閉じる
cv2.destroyAllWindows()
#全ての処理を終了する前にちょっと待つ。
cv2.waitKey(1)

## スケーリング

In [ ]:
import cv2
#画像を読み込む。第二引数は 0:グレースケール、1:カラー画像 -1:カラー＋アルファ
img = cv2.imread('Images/arfukuoka.png',1)

#方法1 :fx,fyを使って縦横の倍率を決める(補間方法はオプション)
res=cv2.resize(img,None,fx=1.3, fy=1.3,interpolation = cv2.INTER_CUBIC)
#方法2 :縦横のサイズを直打ちして指定
res2=cv2.resize(img,(200,300),interpolation = cv2.INTER_CUBIC)

#画像をWindowで表示
cv2.imshow('image',img)
cv2.imshow('method1',res)
cv2.imshow('method2',res2)
#キー入力を待つ
cv2.waitKey(0)
#何かしらのキーを打つとウィンドウを閉じる
cv2.destroyAllWindows()
#全ての処理を終了する前にちょっと待つ。
cv2.waitKey(1)

## 並進(縦横の移動)

In [ ]:
import cv2
import numpy as np
#画像を読み込む。第二引数は 0:グレースケール、1:カラー画像 -1:カラー＋アルファ
img = cv2.imread('Images/arfukuoka.png',1)
height,width,_=img.shape

#並進の行列(tx=10, ty=50)
M = np.float32([[1,0,10],[0,1,50]])
#出力画像のサイズ
dst = cv2.warpAffine(img,M,(width,height))

#画像をWindowで表示
cv2.imshow('image',img)
cv2.imshow('result',dst)
#キー入力を待つ
cv2.waitKey(0)
#何かしらのキーを打つとウィンドウを閉じる
cv2.destroyAllWindows()
#全ての処理を終了する前にちょっと待つ。
cv2.waitKey(1)

## 回転

In [ ]:
import cv2
import numpy as np
#画像を読み込む。第二引数は 0:グレースケール、1:カラー画像 -1:カラー＋アルファ
img = cv2.imread('Images/arfukuoka.png',1)
height,width,_=img.shape

#回転行列(画像の真ん中を軸に60°回転)
M = cv2.getRotationMatrix2D((width//2,height//2),60,1)

#出力画像のサイズ
dst = cv2.warpAffine(img,M,(width,height))

#画像をWindowで表示
cv2.imshow('image',img)
cv2.imshow('result',dst)
#キー入力を待つ
cv2.waitKey(0)
#何かしらのキーを打つとウィンドウを閉じる
cv2.destroyAllWindows()
#全ての処理を終了する前にちょっと待つ。
cv2.waitKey(1)

## アフィン変換(変形)

In [ ]:
import cv2
import numpy as np
#画像を読み込む。第二引数は 0:グレースケール、1:カラー画像 -1:カラー＋アルファ
img = cv2.imread('Images/mesh.png',1)
height,width,_=img.shape

#移動前の点
pts1 = np.float32([[0,0],[width,0],[0,height]])
#上記の点の移動先
pts2 = np.float32([[0,0],[width,30],[0,height]])

#指定した3点×2を可視化するための処理
img2=img.copy();
for i in range(3):
    img2 = cv2.circle(img2,(int(pts1[i,0]),int(pts1[i,1])), 10, (0,0,255),-1)

for i in range(3):
    img2 = cv2.circle(img2,(int(pts2[i,0]),int(pts2[i,1])), 5, (255,0,0),-1)
    
#変形の変換行列
M=cv2.getAffineTransform(pts1,pts2)
#変形
dst=cv2.warpAffine(img,M,(width,height))

#画像をWindowで表示
cv2.imshow('image',img2)
cv2.imshow('result',dst)
#キー入力を待つ
cv2.waitKey(0)
#何かしらのキーを打つとウィンドウを閉じる
cv2.destroyAllWindows()
#全ての処理を終了する前にちょっと待つ。
cv2.waitKey(1)

## 射影変換(変形)

In [ ]:
import cv2
import numpy as np
#画像を読み込む。第二引数は 0:グレースケール、1:カラー画像 -1:カラー＋アルファ
img = cv2.imread('Images/mesh.png',1)
height,width,_=img.shape

#移動前の点
pts1 = np.float32([[0,0],[width,0],[width,height],[0,height]])
#上記の点の移動先
pts2 = np.float32([[30,40],[width-100,30],[width-10,height-50],[20,height-30]])

#指定した4点×2の可視化
img2=img.copy();
for i in range(4):
    img2 = cv2.circle(img2,(int(pts1[i,0]),int(pts1[i,1])), 10, (0,0,255),-1)
for i in range(4):
    img2 = cv2.circle(img2,(int(pts2[i,0]),int(pts2[i,1])), 4, (255,0,0),-1)
    
#射影行列の算出
M=cv2.getPerspectiveTransform(pts1,pts2)
#射影変換
dst=cv2.warpPerspective(img,M,(width,height))

#画像をWindowで表示
cv2.imshow('image',img2)
cv2.imshow('result',dst)
#キー入力を待つ
cv2.waitKey(0)
#何かしらのキーを打つとウィンドウを閉じる
cv2.destroyAllWindows()
#全ての処理を終了する前にちょっと待つ。
cv2.waitKey(1)

## 射影変換で斜め→正面変換

In [ ]:
import cv2
import numpy as np
#画像を読み込む。第二引数は 0:グレースケール、1:カラー画像 -1:カラー＋アルファ
img = cv2.imread('Images/marker.jpg',1)
height,width,_=img.shape

#上記のコードとの違いは点の座標と扱う画像だけ！
#移動前の点
pts1 = np.float32([[100,68],[218,58],[263,168],[114,193]])
#上記の点の移動先
pts2 = np.float32([[0,0],[width,0],[width,height],[0,height]])

#指定した4点×2の可視化
img2=img.copy();
for i in range(4):
    img2 = cv2.circle(img2,(int(pts1[i,0]),int(pts1[i,1])), 10, (0,0,255),-1)
for i in range(4):
    img2 = cv2.circle(img2,(int(pts2[i,0]),int(pts2[i,1])), 4, (255,0,0),-1)
    
#射影行列の算出
M=cv2.getPerspectiveTransform(pts1,pts2)
#射影変換
dst=cv2.warpPerspective(img,M,(width,height))

#マーカの黒枠の太さ(マーカー全体の1/4)
marginH=height//4;
marginW=width//4;
#黒枠の内側だけをクリッピング
dst2=dst[marginH:marginH*3,marginW:marginW*3]
#見やすいように300x300に変換
dst2=cv2.resize(dst2,(300,300),interpolation = cv2.INTER_CUBIC)

#画像をWindowで表示
cv2.imshow('image',img2)
cv2.imshow('result',dst)
cv2.imshow('pattern',dst2)
#キー入力を待つ
cv2.waitKey(0)
#何かしらのキーを打つとウィンドウを閉じる
cv2.destroyAllWindows()
#全ての処理を終了する前にちょっと待つ。
cv2.waitKey(1)

# マウス操作に対応

In [ ]:
import cv2
import numpy as np

index=0

# マウス操作に関する処理
def mousePos(event,x,y,flags,param):
    global index
    #index番目(0~3)
    if index<3:
        #マウスの位置をindex番目の点の表にする
        pts2[index,0]=x
        pts2[index,1]=y
    #マウスをクリックしたら適当なindexにする
    if event == cv2.EVENT_LBUTTONUP:
        index=100

#画像を読み込む。第二引数は 0:グレースケール、1:カラー画像 -1:カラー＋アルファ
img = cv2.imread('Images/mesh.png',1)
height,width,_=img.shape

#移動前後の点(マウスで動かすから適当でOK)
pts1 = np.float32([[0,0],[width,0],[0,height]])
pts2 = np.float32([[0,0],[width,10],[0,height]])

#表示する前にwindow名をつける
cv2.namedWindow('image2')
#上記windowにマウスアクションを対応させる
cv2.setMouseCallback('image2',mousePos)
 
while (True):
    #指定した点を表示
    img2=img.copy();
    for i in range(3):
        img2 = cv2.circle(img2,(int(pts1[i,0]),int(pts1[i,1])), 10, (0,0,255),-1)
    for i in range(3):
        img2 = cv2.circle(img2,(int(pts2[i,0]),int(pts2[i,1])), 4, (255,0,0),-1)
    #アフィン変換
    M=cv2.getAffineTransform(pts1,pts2)
    dst=cv2.warpAffine(img,M,(width,height))
    
    #画像をWindowで表示
    cv2.imshow('image2',img2)
    cv2.imshow('dst',dst)
    #30ミリ秒、キー入力を待つ
    key = cv2.waitKey(30) & 0xFF
    #qを入力した場合は終了
    #0~2を押すと対応する点がマウスに追従するようになる。
    #画面をクリックしたらマウスに追従しなくなる。(mousePos内参照)
    if key ==ord('q') :
        break
    elif key == ord('0'):
        index=0
    elif key == ord('1'):
        index=1
    elif key==ord('2'):
        index=2
    
        
#キャプチャを終了
cv2.waitKey(1)
#全てのウィンドウを閉じる
cv2.destroyAllWindows()
#フリーズ対策として終わる前にちょっと待つ
cv2.waitKey(1)